# Import

In [4]:
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

# Data

In [5]:
import os
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

def convert_to_sparse_matrix(docs):
    indptr = [0]
    indices = []
    data = []
    vocabulary = {}
    for d in docs:
        for term in d:
            index = vocabulary.setdefault(term, len(vocabulary))
            indices.append(index)
            data.append(1)
        indptr.append(len(indices))
    return csr_matrix((data, indices, indptr), dtype=int)


data_dir = 'messages'
messages = []
y = []
for filename in os.listdir(data_dir):
    with open(os.path.join(data_dir, filename)) as file:
        subject = file.readline()
        skip = file.readline()
        content = file.readline()
        messages.append(content.split())
    if "legit" in filename:
        y.append("legit")
    else:
        y.append("spam")

X = convert_to_sparse_matrix(messages)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


print(X_train, X_test, y_train, y_test)

  (0, 3582)	1
  (0, 39)	1
  (0, 839)	1
  (0, 76)	1
  (0, 2967)	1
  (0, 122)	1
  (0, 2241)	1
  (0, 2736)	1
  (0, 94)	1
  (0, 1001)	1
  (0, 145)	1
  (0, 16731)	1
  (0, 42)	1
  (0, 540)	1
  (0, 45)	1
  (0, 24)	1
  (0, 1223)	1
  (0, 553)	1
  (0, 18)	1
  (0, 842)	1
  (0, 24)	1
  (0, 325)	1
  (0, 582)	1
  (0, 32)	1
  (0, 223)	1
  :	:
  (729, 611)	1
  (729, 611)	1
  (729, 611)	1
  (729, 611)	1
  (729, 611)	1
  (729, 611)	1
  (729, 611)	1
  (729, 611)	1
  (729, 611)	1
  (729, 611)	1
  (729, 611)	1
  (729, 611)	1
  (729, 611)	1
  (729, 611)	1
  (729, 611)	1
  (729, 611)	1
  (729, 611)	1
  (729, 611)	1
  (729, 611)	1
  (729, 611)	1
  (729, 611)	1
  (729, 611)	1
  (729, 611)	1
  (729, 611)	1
  (729, 611)	1   (0, 196)	1
  (0, 11)	1
  (0, 848)	1
  (0, 48)	1
  (0, 17748)	1
  (0, 1713)	1
  (0, 9827)	1
  (0, 665)	1
  (0, 661)	1
  (0, 11)	1
  (0, 663)	1
  (0, 4)	1
  (0, 1838)	1
  (0, 669)	1
  (0, 1713)	1
  (0, 718)	1
  (0, 5805)	1
  (0, 55)	1
  (0, 1540)	1
  (0, 122)	1
  (0, 2272)	1
  (0, 748)	1
  (0, 

# Naive bayes

In [6]:
def get_class(spam_prob, threshold=0.5):
    if spam_prob < threshold:
        return "legit"
    else:
        return "spam"

In [7]:
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [8]:
clf.score(X_test, y_test)

0.9583333333333334

In [9]:
pred_prob = clf.predict_proba(X_test)

In [10]:
pred_classes = pd.Series(pred_prob[:,  1]).apply(get_class)

In [11]:
confusion_matrix(y_test, pred_classes)[0,1]

9

In [12]:
thresholds = np.arange(0, 1, 0.01)

In [13]:
min_error = [1000]
thres = []
for i in thresholds:
    def get_class(spam_prob, threshold=i):
        if spam_prob < threshold:
            return "legit"
        else:
            return "spam"
    
    pred_classes = pd.Series(pred_prob[:,  1]).apply(get_class)
    if confusion_matrix(y_test, pred_classes)[0,1] < min_error[-1]:
        min_error.append(confusion_matrix(y_test, pred_classes)[0,1])
        thres.append(i)

In [14]:
min_error

[1000, 199, 14, 13, 12, 11, 10, 9, 8]

In [15]:
thres

[0.0, 0.01, 0.06, 0.09, 0.12, 0.21, 0.37, 0.79]

In [17]:
confusion_matrix(y_test, pred_classes)

array([[191,   8],
       [  7, 154]])